# Event Viewer

## Imports

The first line sets the backend used by matplotlib. To be able to interact with the plots, make it say `%matplotlib osx`. For inline plots, use `%matplotlib inline`.

In [ ]:
%matplotlib inline

In [ ]:
import sys
sys.path.append('..')
import pytpc
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy
import scipy.stats as ss
import seaborn as sns

## Loading Files

The event file should always be loaded.

In [ ]:
pads = pytpc.generate_pad_plane(rotation_angle=-108.*pytpc.pi/180.)
ef = pytpc.EventFile("/Users/josh/Documents/Data/Alphas-Dec14/run_0232_ps.evt")
#ef = evtdata.EventFile("/Users/josh/Documents/Data/Alphas-Dec14/OldEvts/run_0048.evt")

The files below should only be loaded if using event files without background subtraction.

In [ ]:
peds = evtdata.load_pedestals("/Users/josh/Dropbox/routing/Peds20141208-2.csv")
pm = evtdata.load_padmap("/Users/josh/Dropbox/routing/Lookup20141208.csv")

## Reading an Event

This one reads the event from the file:

In [ ]:
evt = ef[105]

If pedestals have not been subtracted already in the file, then run this cell:

In [ ]:
evt.traces['data'] -= [ numpy.ones(512,) * peds[tuple(addr)] for addr in evt.traces[['cobo','asad','aget','channel']] ]
evt.traces['data'] = ss.threshold(evt.traces['data'], threshmin=35)
evt.traces['pad'] = [ pm[tuple(addr)] for addr in evt.traces[['cobo','asad','aget','channel']] ]

## Plots

### Pad Plane

In [ ]:
p = pytpc.pad_plot(evt.hits(), pads)

### 3D Plot

In [ ]:
cp = pytpc.chamber_plot(evt.xyzs(drift_vel=1, clock=12.5))

### Mesh Plot

In [ ]:
mesh = evt.traces['data'].sum(0)
plt.plot(mesh)
plt.xlabel('Time Bucket')
plt.ylabel('Activation')
plt.xlim(0, 512)

### Individual Channels

In [ ]:
lines = evt.traces['data']
for l in lines:
    plt.plot(l)
    plt.xlim(0, 512)
plt.xlabel('Time Buckets')
plt.ylabel('Activation [arb. units]')
plt.savefig('/Users/josh/Desktop/all_traces.png', dpi=200)

In [ ]:
sns.set_context('talk')

In [ ]:
sns.set_style('darkgrid')